# Imports

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt

# Preprocessing

In [ ]:
df = pd.read_csv("logs.csv", sep=",")

In [ ]:
df["Door_open"] = df["Door"]=="open"
df["Door_ajar"] = df["Door"]=="ajar"
df.drop(columns=["Door"], axis=1, inplace=True)
df["t"] = pd.to_datetime(df["t"], format="mixed")

df["Alarm level"] = df["Alarm level"].map(lambda x: int(x[:1]))

In [ ]:
#filtered_df = df.loc[(df['t'] >= '2025-09-18') & (df['t'] < '2025-09-23')]

## Plot ajar door

In [ ]:
ajar_df = df[["t" ,"Door_ajar", "Alarm level"]].copy()
#ajar_df["Door"] = ajar_df["Door"].apply(lambda x: 1 if x == "ajar" else 0) # make numeric for processing

ajar_df["Door_ajar"] = np.where(ajar_df["Door_ajar"] & (ajar_df["Alarm level"] > 2), 1, 0) # filter for slightly open door and alarm level (repeated measurements)

# find beginnings (find all [0,1] occurrences)
idx = np.where(
    np.all(np.lib.stride_tricks.sliding_window_view(ajar_df["Door_ajar"].copy(), 2) == [0,1], axis=1)
)[0] + 1 # +1 so that we don't find the 0s

beginnings = np.zeros((len(ajar_df)))
beginnings[idx] = 1
ajar_df["Door_ajar_start"] = beginnings

plt.figure(figsize=(10,10))
ax = ajar_df.plot("t", "Door_ajar_start")
plt.title("Alarm triggered because door was slightly left open")
plt.ylabel("alarm triggered")
plt.xlabel("time")
ax.get_legend().remove()
plt.yticks(ticks=[0,1], labels=["NO", "YES"])
plt.show()
print("Total times alarm was triggered because door was slightly left open:", int(np.sum(beginnings)))

In [ ]:
tmp = ajar_df.copy()
tmp["t"] = ajar_df["t"].apply(lambda t: t.day)
tmp = tmp.groupby("t")["Door_ajar_start"].sum()
print(tmp)

## Plot open door

In [ ]:
# plot open door
open_df = df[["t" ,"Door_open", "Alarm level"]].copy()
open_df["Door_open"] = open_df["Door_open"].apply(lambda x: 1 if x == "open" else 0) #todo
open_df["door_num"] = np.where((open_df["Door_open"] == 1) & (tmp["Alarm level"] > 0), 1, 0)
open_df.plot("t", "door_num")

In [ ]:
a = np.array([1, 2, 3, 2, 3, 4, 5])
b = np.array([2, 3])

idx = np.where(
    np.all(np.lib.stride_tricks.sliding_window_view(a, len(b)) == b, axis=1)
)[0]
print(idx)